In [36]:
!pip install mdtraj
!pip install msmbuilder

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.9 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
  Using cached msmbuilder-3.8.0.tar.gz (304 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [334 lines of output]
      C compiler:
      Using built-in specs.
      COLLECT_GCC=/usr/bin/gcc
      COLLECT_LTO_WRAPPER=/usr/libexec/gcc/x86_64-redhat-linux/8/lto-wrapper
      OFFLOAD_TARGET_NAMES=nvptx-none
      OFFLOAD_TARGET_DEFAULT=1
      Target: x86_64-redhat-linux
      Configured with: ../configure --enable-bootstrap --enable-languages=c,c++,fortran,lto --prefix=/usr --mandir=/usr/share/man --infodir=/usr/share/i

In [16]:
from ase.io import read, write


from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np

In [28]:




# Load ASE trajectory (e.g., extxyz format)
traj = read('../../data/md/geometry_2_opt_fixed_md_tblite.extxyz', index=':')

# Extract Cartesian coordinates of all frames
n_frames = len(traj)
n_atoms = len(traj[0])
coordinates = np.zeros((n_frames, n_atoms * 3))

for i, atoms in enumerate(traj):
    coordinates[i, :] = atoms.get_positions().flatten()

# Apply PCA to the coordinates
pca = PCA(n_components=6)  # Reduce dimensionality to 2 for easy visualization
reduced_coords = pca.fit_transform(coordinates)

# Now reduced_coords contains the trajectory in a 2D space of the first two PCs
# You can apply clustering to find representative frames (anchors)

# Example: Use KMeans to select representative frames


n_clusters = int(np.ceil(np.min([50, np.sqrt(len(traj))])))
print(f'Selecting {n_clusters} anchors')
kmeans = KMeans(n_clusters=n_clusters).fit(reduced_coords)

# Get indices of the representative frames (closest to the cluster centers)
representative_indices = []
for cluster in range(n_clusters):
    indices = np.where(kmeans.labels_ == cluster)[0]
    # Find the frame closest to the centroid
    centroid = kmeans.cluster_centers_[cluster]
    closest_frame = indices[np.argmin(np.linalg.norm(reduced_coords[indices] - centroid, axis=1))]
    representative_indices.append(closest_frame)

# Output the representative frames (anchors)
print("Representative frames (anchor points):", representative_indices)

# You can now use these indices to extract and save the anchor frames
anchors = [traj[i] for i in representative_indices]

write('anchors.xyz', anchors)


Selecting 26 anchors
Representative frames (anchor points): [653, 221, 439, 77, 331, 536, 168, 31, 588, 277, 385, 489, 124, 358, 249, 560, 619, 412, 54, 11, 304, 512, 192, 148, 465, 101]


In [34]:
np.where(kmeans.labels_ == 0)[0]

array([634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646,
       647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675])